# HW03 - Fine-tuning on Animal Dataset

This notebook implements fine-tuning of a pre-trained model on my custom animal dataset (cats, dogs, rabbits, capybaras, owls).

**Dataset**: Animals dataset with 5 classes
**Task**: Fine-tune a pre-trained model for image classification

## 1. Import Libraries

In [ ]:
from fastai.vision.all import *
from pathlib import Path
import warnings

# Ignore palette warnings
warnings.filterwarnings(
    "ignore",
    message="Palette images with Transparency expressed in bytes should be converted to RGBA images",
    category=UserWarning
)

## 2. Data Preparation

Load the dataset that was prepared in the previous notebooks. The dataset is already organized in the correct train/test structure.

In [ ]:
# Path to the dataset (using the animals_dataset created in previous notebooks)
path = Path('./animals_dataset/train')

# Create data loaders
dls = ImageDataLoaders.from_folder(
    path,                 # Path to the train folder
    train='.',            # Use the entire train folder for splitting
    valid_pct=0.2,        # 20% of the data will be used for validation
    seed=42,              # Set a seed for reproducibility
    item_tfms=Resize(460),# Resize the images to 460x460
    batch_tfms=aug_transforms(size=224, min_scale=0.75), # Data augmentation 
    bs=32                 # Batch size 32 (reduced for better performance)
)

print(f"Number of classes: {len(dls.vocab)}")
print(f"Classes: {dls.vocab}")
print(f"Training samples: {len(dls.train_ds)}")
print(f"Validation samples: {len(dls.valid_ds)}")

## 3. Show Sample Data

In [ ]:
# Show a batch of images
dls.show_batch(max_n=9, figsize=(10, 10))

## 4. Model Selection and Learning Rate Finding

Using ResNet18 as the base model - it's a good balance between performance and speed for this task.

In [ ]:
# Create a vision learner with ResNet18
learn = vision_learner(dls, resnet18, metrics=error_rate)

# Find the optimal learning rate
learn.lr_find()

## 5. Fine-tuning the Model

Based on the learning rate finder, I'll set an appropriate learning rate and train the model.

In [ ]:
# Set optimal learning rate (adjust based on lr_find results)
lr = 1e-3  # This is a good starting point, adjust based on lr_find plot
epochs = 5  # Number of epochs for fine-tuning

print(f"Training with learning rate: {lr} for {epochs} epochs")

In [ ]:
# Create a fresh learner and fine-tune
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(epochs, lr)

## 6. Model Evaluation

Let's evaluate how well our model performs on the validation set.

In [ ]:
# Create interpretation object
interp = ClassificationInterpretation.from_learner(learn)

# Plot confusion matrix
interp.plot_confusion_matrix(figsize=(8, 8), dpi=80)

## 7. Analyze Misclassifications

In [ ]:
# Show the most confused/misclassified images
interp.plot_top_losses(9, figsize=(13, 13))

## 8. Test on Test Set

Now let's test our model on the separate test set that we prepared.

In [ ]:
# Create a test DataLoader
test_path = Path('./animals_dataset/test')
test_dl = learn.dls.test_dl(get_image_files(test_path))

print(f"Test set size: {len(test_dl.items)}")

In [ ]:
# Get predictions on test set
preds, targets = learn.get_preds(dl=test_dl)
pred_labels = torch.argmax(preds, dim=1)

# Get class names
class_names = learn.dls.vocab

# Convert predictions to class names
pred_class_names = [class_names[i] for i in pred_labels]
true_class_names = [Path(t).parent.name for t in test_dl.items]

print(f"Sample predictions:")
for i in range(min(10, len(pred_class_names))):
    status = "✓" if true_class_names[i] == pred_class_names[i] else "✗"
    print(f"{status} True: {true_class_names[i]:10} | Predicted: {pred_class_names[i]:10}")

In [ ]:
# Calculate test accuracy
from statistics import mean

test_accuracy = mean([actual == predicted for actual, predicted in zip(true_class_names, pred_class_names)])
test_error_rate = 1 - test_accuracy

print(f"\n🎯 TEST RESULTS:")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"Test Error Rate: {test_error_rate:.4f} ({test_error_rate*100:.2f}%)")

# Per-class accuracy
print(f"\n📊 PER-CLASS RESULTS:")
for class_name in class_names:
    class_true = [i for i, true_class in enumerate(true_class_names) if true_class == class_name]
    if len(class_true) > 0:
        class_correct = sum([true_class_names[i] == pred_class_names[i] for i in class_true])
        class_accuracy = class_correct / len(class_true)
        print(f"{class_name:10}: {class_accuracy:.4f} ({class_accuracy*100:.2f}%) - {class_correct}/{len(class_true)} correct")

## 9. Save the Model

Export the trained model for later use in Gradio app.

In [ ]:
# Export the model
model_path = 'animal_classifier_model.pkl'
learn.export(model_path)

print(f"✅ Model saved as: {model_path}")
print(f"📁 Model file size: {Path(model_path).stat().st_size / (1024*1024):.2f} MB")
print(f"\n🚀 This model is ready for deployment in Gradio!")

## 10. Model Summary

Let's summarize what we've accomplished.

In [ ]:
print("🎉 FINE-TUNING COMPLETE!")
print("=" * 50)
print(f"📊 Dataset: 5-class animal classifier")
print(f"🏗️  Model: ResNet18 (fine-tuned)")
print(f"📝 Classes: {', '.join(class_names)}")
print(f"🎯 Training completed successfully!")
print(f"💾 Model exported for Gradio deployment")
print("=" * 50)
print(f"\n📋 NEXT STEPS:")
print(f"1. ✅ Fine-tuning completed")
print(f"2. 🔄 Create Gradio app for deployment")
print(f"3. 🚀 Deploy to HuggingFace Spaces")